In [1]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import plotly.express as px

In [2]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSlevels_df_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']

In [3]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url, dtype=None, parse_dates=None):
    req = urllib.request.Request(url)
    req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req), dtype=dtype, parse_dates=parse_dates)

ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all", dtype=dtypes)
ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life', 'electric_quadrupole']]
ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
ground_states_df = ground_states_df.copy()
ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
ground_states_df = ground_states_df.set_index(['z', 'a'], drop=True)

In [4]:
levels_df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


levels_df.loc[levels_df['half_life'] == 'STABLE', 'half_life'] = 'inf'
levels_df['half_life'] = levels_df['half_life'].astype('Float64')

levels_df['a'] = levels_df['z'] + levels_df['n']

levels_df = levels_df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
levels_df['energy'] = levels_df['energy'] / 1000 #MeV
levels_df['ripl_shift'] = levels_df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

levels_df = levels_df.set_index(['z', 'a'], drop=True)

In [5]:
# Filter out unknown energies

levels_df = levels_df[levels_df['energy_shift'].isna()]
levels_df = levels_df.drop(['energy_shift', 'ripl_shift'], axis='columns')

In [6]:
# Keep only even-even nuclei

df = levels_df[(levels_df.index.get_level_values('z') % 2 == 0) & (levels_df.index.get_level_values('a') % 2 == 0)]

In [7]:
# Filter out uncertain jp

df = df[df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

df['j'] = df['jp'].str[:-1]
df['p'] = df['jp'].str[-1]

odd_spins = df['j'].str.contains('/')
even_spins = ~odd_spins

df.loc[odd_spins,'j_float'] = df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
df.loc[even_spins,'j_float'] = df[even_spins]['j'].astype(float)

df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
df.loc[even_spins,'j_evenness'] = (df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)

df['p_bit'] = df['p'].str.fullmatch(r'\-').astype(int)

In [8]:
# Only consider first even levels i.e. 0^+_1, 2^+_1, 4^+_1, 6^+_1, 8^+_1, ...

first_even_levels = df[(df['jp_order'] == 1) & df['j_evenness'].fillna(False) & (df['p_bit'] == 0)].copy()

# Number of quanta of quadrupole oscillation
first_even_levels['quanta'] = (first_even_levels['j_float'].astype(int) // 2)

# Remove extraenous information
first_even_levels = first_even_levels.drop(columns=['idx', 'jp', 'jp_order', 'j', 'p', 'j_float', 'j_evenness', 'p_bit'])

In [9]:
# Filter out not enough levels
first_even_levels = first_even_levels.loc[first_even_levels.groupby(level=first_even_levels.index.names).size() >= 4]

In [18]:
# def get_ratios(group):
#     # Calculate ratio of subsequent energy values
#     energy_ratios = group['energy'].iloc[1:].values / group['energy'].iloc[:-1].values
#     return energy_ratios

def get_ratios(group):
    first = group['energy'].iloc[2] / group['energy'].iloc[1]
    second = group['energy'].iloc[3] / group['energy'].iloc[2]

    results = pd.DataFrame({'first_ratio': first, 'second_ratio': second}, index=group.index)
    return results

In [21]:
first_even_levels_groups = first_even_levels.groupby(level=df.index.names, as_index=False)

ratios = first_even_levels_groups.apply(lambda group: get_ratios(group)).droplevel(0)

In [26]:
merged = first_even_levels.merge(ratios, how='left', on=['z','a'])

nuclides = merged.groupby(level=merged.index.names, as_index=True).head(1)[['symbol', 'n', 'first_ratio', 'second_ratio']]

nuclides

symbol    n  first_ratio  second_ratio
z  a                                         
8  16       O    8     1.497159      1.430601
   18       O   10     1.793499      3.288474
10 20      Ne   10     2.600090      2.066436
12 24      Mg   12     3.012313      1.967861
   26      Mg   14     2.387789      2.109616
...       ...  ...          ...           ...
80 200     Hg  120     2.574429      1.801787
   202     Hg  122     2.547775      1.775875
   204     Hg  124     2.584411      1.941989
82 206     Pb  124     2.096982      1.936116
   208     Pb  126     1.058359      1.023058

[126 rows x 4 columns]

In [29]:
mean = nuclides['first_ratio'].mean()

percentages = [0,25, 50, 75, 100]
quantiles = [nuclides['first_ratio'].quantile(percentage/100) for percentage in percentages]

quantile_counts = [(nuclides['first_ratio'] <= quantile).sum() for quantile in quantiles]
# If sorting as oscillators.sort_values(by='first_ratio', ascending=True).iloc[idx]['first_ratio']
# values up to and excluding idx=quantile_count[i] will have values up to an including quantiles[i]

percentages = [quantile_count / len(nuclides['first_ratio']) * 100 for quantile_count in quantile_counts]

fig = px.histogram(nuclides, x='first_ratio', nbins=100, title='Ratio of E4+ to E2+')
fig.update_layout(
    xaxis_title='E4+/E2+',
    yaxis_title='Count',
    width=800,
    showlegend=False
)

for percentage, quantile in zip(percentages, quantiles):
    print(f"{percentage:.1f}% have E4+/E2+ < {quantile:.3f}")
    fig.add_vline(x=quantile, line_dash="dash", line_color="red", annotation_text=f"{percentage:.1f}%")

fig.show()

0.8% have E4+/E2+ < 1.058
25.4% have E4+/E2+ < 1.994
50.0% have E4+/E2+ < 2.340
74.6% have E4+/E2+ < 2.599
100.0% have E4+/E2+ < 3.310


In [31]:
fig = px.scatter(nuclides.reset_index(), 
          x='n',
          y='z', 
          color='first_ratio',
          title='Ratio of E4+ to E2+ For Even-Even Nuclei',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'first_ratio': 'E4+/E2+'},
        #   color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          color_continuous_scale='magma',
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()

In [95]:
def get_nuclide_type(nuclide, atol=0.2):
    if nuclide['first_ratio'] < 2-atol:
    # if np.isclose(nuclide['first_ratio'], 1, atol=0.5):
        return '<2.0: low ratio'
    elif np.isclose(nuclide['first_ratio'], 2, atol=atol):
        return '~2.0: harmonic oscillator'
    elif np.isclose(nuclide['first_ratio'], 2.5, atol=atol):
        return '~2.5: axially asymmetric rotor'
    elif np.isclose(nuclide['first_ratio'], 3.33, atol=atol):
        return '~3.3 axially symmetric rotor'
    else:
        return 'unknown'
nuclides['type'] = [get_nuclide_type(nuclides.iloc[i]) for i in range(len(nuclides))]
nuclides['type'] = nuclides['type'].astype('category')
nuclides

symbol    n  first_ratio  second_ratio                            type
z  a                                                                         
8  16       O    8     1.497159      1.430601                 <2.0: low ratio
   18       O   10     1.793499      3.288474                 <2.0: low ratio
10 20      Ne   10     2.600090      2.066436  ~2.5: axially asymmetric rotor
12 24      Mg   12     3.012313      1.967861                         unknown
   26      Mg   14     2.387789      2.109616  ~2.5: axially asymmetric rotor
...       ...  ...          ...           ...                             ...
80 200     Hg  120     2.574429      1.801787  ~2.5: axially asymmetric rotor
   202     Hg  122     2.547775      1.775875  ~2.5: axially asymmetric rotor
   204     Hg  124     2.584411      1.941989  ~2.5: axially asymmetric rotor
82 206     Pb  124     2.096982      1.936116       ~2.0: harmonic oscillator
   208     Pb  126     1.058359      1.023058                 <2.0: low ratio

[126 rows x 5 columns]

In [99]:
colors = px.colors.sequential.Viridis

fig = px.scatter(nuclides.reset_index(), 
          x='n',
          y='z', 
          color='type',
          title='Ratio of E4+ to E2+ For Even-Even Nuclei',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'first_ratio': 'E4+/E2+ type'},
          color_discrete_sequence=[colors[0], colors[7], "white", colors[4], colors[9]],
          height=600,
          width=700)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()